In [ ]:
%%capture
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import BertModel,default_data_collator, AutoTokenizer, Trainer, TrainingArguments,BertTokenizer,AutoModelForSequenceClassification
from tqdm import tqdm
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:


from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd gdrive/MyDrive/ASU/Self_Work/

[Errno 2] No such file or directory: 'gdrive/MyDrive/ASU/Self_Work/'
/content/gdrive/MyDrive/ASU/Self_Work


In [ ]:
pwd

'/content/gdrive/My Drive/ASU/Self_Work'

### Data Processing

In [ ]:
from datasets import load_dataset

boolq_dataset = load_dataset("super_glue","boolq")

Reusing dataset super_glue (/root/.cache/huggingface/datasets/super_glue/boolq/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
boolq_dataset['train']

Dataset({
    features: ['question', 'passage', 'idx', 'label'],
    num_rows: 9427
})

In [ ]:
boolq_dataset['train'][0]

{'idx': 0,
 'label': 1,
 'passage': 'Persian language -- Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.',
 'question': 'do iran and afghanistan speak the same language'}

In [ ]:
model_ckpt='bert-base-cased'
MAX_LEN = 100
BATCH_SIZE=16
EPOCHS = 4
LEARNING_RATE = 3e-5
doc_stride = 128 


In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_ckpt)
#tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
boolq_enc = boolq_dataset.map(lambda x : tokenizer(x['question'], x['passage'], truncation="only_second"), batched=True)

HTTPError: ignored

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt)

In [ ]:

args = TrainingArguments(
    f"test-boolq",
    evaluation_strategy = "epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS, # Set num_train_epochs to 1 as test
    weight_decay=0.01,
)

from transformers import default_data_collator
data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=boolq_enc["train"],
    eval_dataset=boolq_enc["validation"],
    # data_collator=data_collator, # I had to comment this because it was not working with the default collator
    tokenizer=tokenizer,
)

trainer.train()